In [ ]:
import cv2
import sys
import http.client, urllib.request, urllib.parse, urllib.error, base64
import json
from google.cloud import storage
from google.oauth2 import service_account

ENDPOINT = 'realeyes.cognitiveservices.azure.com'
credentials = service_account.Credentials.from_service_account_file(
        '/Users/wangyyt_1013/workspace/dubhacks19/eminent-crane-255805-c1f4cf527229.json')
client = storage.Client(
    credentials = credentials,
    project=credentials.project_id)

bucket = client.bucket('dubhacks19')
headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': 'e4769d386b394c5fb880c5ba2b499f56',
}
faceCascade = cv2.CascadeClassifier('/Users/wangyyt_1013/workspace/dubhacks19/haarcascade_frontalface_default.xml')
names = ['Elaine', 'Chenoa', 'Caroline', 'Bolaji']

for name in names:
    # for adding images to training set-------------------------------------------------------------------------------
    video_capture = cv2.VideoCapture("/Users/wangyyt_1013/workspace/dubhacks19/" + name.lower() + "1.avi")

    # creating the person
    person_group = "1" # sourcing user input
    description = "The nicest person ever." # sourcing user input

    params_create_person = urllib.parse.urlencode({
    })

    body_create_person = json.dumps({
        "name": name,
        "userData": description
    })
    try:
        conn = http.client.HTTPSConnection(ENDPOINT)
        conn.request("POST", "/face/v1.0/persongroups/" + person_group + "/persons?%s" % params_create_person, body_create_person, headers)
        response = conn.getresponse()
        data = response.read() # user id is returned
        print(data)
        person_id = json.loads(data)['personId'] # storing the returned person_id
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

    image_counter = 0
    # adding the images to the person
    while (video_capture.isOpened()):
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        if ret == False:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, 1.3, 5)

        # upload image to cloud storage and get url
        file_path = '../images/' + str(person_id) + '-' + str(image_counter) + '.jpg'
        destination_path = 'person-group' + str(person_group) + '/' + name + '/' + str(image_counter) + '.jpg'

        cv2.imwrite(file_path, frame)
        blob = bucket.blob(destination_path)
        blob.upload_from_filename(file_path)
        url = blob.public_url

        for (x, y, w, h) in faces:
            params_add_image = urllib.parse.urlencode({
                # Request parameters
                # 'targetFace': str(x) + "," + str(y) + "," + str(w) + "," + str(h),
            })
            body_add_image = json.dumps({
                'url': url
            })
            try:
                conn = http.client.HTTPSConnection(ENDPOINT)
                conn.request("POST", "/face/v1.0/persongroups/" + person_group + "/persons/" + \
                             person_id + "/persistedFaces?%s" % params_add_image, body_add_image, headers)
                response = conn.getresponse()
                data = response.read()
                print(data)
                conn.close()
            except Exception as e:
                print("[Errno {0}] {1}".format(e.errno, e.strerror))

            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.imshow('Video', frame)
        image_counter += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything is done, release the capture
    video_capture.release()
    cv2.destroyAllWindows()

b'{"personId":"631749af-b99d-4882-8849-dfeb2d2eca3e"}'
b'{"persistedFaceId":"1a98cb87-9cfd-492a-ae3d-94291d258b8a"}'
b'{"persistedFaceId":"f433d74f-2f11-4bcd-af39-9c178800be95"}'
b'{"persistedFaceId":"b32842d6-31fe-4582-a10f-9bd40a72744a"}'
b'{"persistedFaceId":"a9dbe21d-253d-4b2c-874e-9980f92fccda"}'
b'{"persistedFaceId":"34758d66-fe66-45ba-a534-aca04280cf30"}'
b'{"persistedFaceId":"e572cd1c-2222-4149-b8b1-4e5bd8ae1db1"}'
b'{"persistedFaceId":"e9339787-b30c-42a9-8ee3-f364d6dc678d"}'
b'{"persistedFaceId":"1431ae38-a96c-4053-b121-7ccbc6e10519"}'
b'{"persistedFaceId":"85f873bc-6b6b-4e16-9db4-3b358d717805"}'
b'{"persistedFaceId":"6b219b81-da94-4a86-9408-951988a5a8ba"}'
b'{"persistedFaceId":"5193a005-db8a-4565-955d-d3d8d9b46f0f"}'
b'{"persistedFaceId":"9ef6404d-5232-4840-aff7-c7df19751b58"}'
b'{"persistedFaceId":"88301e9f-29ea-45ec-9d52-1b06c154cc4b"}'
b'{"persistedFaceId":"73c0da68-6082-477a-8d6f-210e31508240"}'
b'{"persistedFaceId":"e6e90a20-f767-49cb-b8c0-b35166c6303a"}'
b'{"persistedFa

In [ ]:
import cv2
import sys
import http.client, urllib.request, urllib.parse, urllib.error, base64
import json
from google.cloud import storage
from google.oauth2 import service_account

ENDPOINT = 'realeyes.cognitiveservices.azure.com'
credentials = service_account.Credentials.from_service_account_file(
        '/Users/wangyyt_1013/workspace/dubhacks19/eminent-crane-255805-c1f4cf527229.json')
client = storage.Client(
    credentials = credentials,
    project=credentials.project_id)

bucket = client.bucket('dubhacks19')
faceCascade = cv2.CascadeClassifier('/Users/wangyyt_1013/workspace/dubhacks19/haarcascade_frontalface_default.xml')
names = ['Elaine', 'Chenoa', 'Caroline', 'Bolaji']


person_group = "1"
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': 'e4769d386b394c5fb880c5ba2b499f56',
}
params = urllib.parse.urlencode({
})
body = json.dumps({
})


try:
    conn = http.client.HTTPSConnection(ENDPOINT)
    conn.request("POST", "/face/v1.0/persongroups/" + person_group + "/train?%s" % params, body, headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))



In [ ]:
# testing with detect API
for name in names:
    video_capture = cv2.VideoCapture("/Users/wangyyt_1013/workspace/dubhacks19/" + name.lower() + "2.avi")
    
    image_counter = 0
    while (video_capture.isOpened()):
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        if ret == False:
            break
        
        # getting the public url for the image
        file_path = '../test/' + str(image_counter) + '.jpg'
        destination_path = 'person-group' + str(person_group) + '/test/' + str(image_counter) + '.jpg'

        cv2.imwrite(file_path, frame)
        blob = bucket.blob(destination_path)
        blob.upload_from_filename(file_path)
        url = blob.public_url
        
        # detecting faces in the image
        headers = {
            # Request headers
            'Content-Type': 'application/json',
            'Ocp-Apim-Subscription-Key': 'e4769d386b394c5fb880c5ba2b499f56',
        }
        params = urllib.parse.urlencode({
        })
        body = json.dumps({
            "url": url
        })
        
        try:
            conn = http.client.HTTPSConnection(ENDPOINT)
            conn.request("POST", "/face/v1.0/detect?%s" % params, body, headers)
            response = conn.getresponse()
            face_ids = response.read()
            print(face_ids)
            conn.close()
        except Exception as e:
            print("[Errno {0}] {1}".format(e.errno, e.strerror))

        body = json.dumps({
            "personGroupId": person_group,
            "faceIds": face_ids[0].faceId,
            "maxNumOfCandidatesReturned": 1,
            "confidenceThreshold": 0.5
        })
        
        try:
            conn = http.client.HTTPSConnection(ENDPOINT)
            conn.request("POST", "/face/v1.0/identify?%s" % params, body, headers)
            response = conn.getresponse()
            data = response.read()
            print("prediction result:", data)
            conn.close()
        except Exception as e:
            print("[Errno {0}] {1}".format(e.errno, e.strerror))
            
        image_counter += 1
